# 🎨 Stable Diffusion Image Generation Demo

This notebook demonstrates how to generate images using the **Stable Diffusion v1.5** model via Hugging Face's `diffusers` library.

You can input your own text prompts, and the model will generate high-quality images based on those descriptions.

In [1]:
# Install requirements
# !pip install -r requirements.txt

In [2]:
# Imports
import torch
from diffusers import StableDiffusionPipeline
import os
from datetime import datetime
from PIL import Image
import configparser

/Users/suvz47/Code/Cursor/NIAT Demo/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Setup Config

In [3]:
def load_config():
    """
    Load configuration from config.ini file.
    
    Returns:
        configparser.ConfigParser: Configuration object
    """
    config = configparser.ConfigParser()
    config_path = '../config.ini'
    
    if os.path.exists(config_path):
        config.read(config_path)
        print("Configuration loaded from config.ini")
    else:
        print("config.ini not found")
    
    return config

config = load_config()

Configuration loaded from config.ini


## Setup the Stable Diffusion Pipeline

In [4]:
def setup_pipeline(config):
    """
    Initialize and configure the Stable Diffusion pipeline.
    
    Args:
        config: Configuration object from config.ini
    
    Returns:
        StableDiffusionPipeline: Configured pipeline ready for image generation
    """
    print("Loading Stable Diffusion model...")
    
    # Get model paths from config
    local_model_path = config.get('Model', 'local_model_path')
    huggingface_model = config.get('Model', 'huggingface_model')
    
    # Check if local model exists
    if os.path.exists(local_model_path):
        print(f"Using local model from: {local_model_path}")
        # Load the model from local path
        pipe = StableDiffusionPipeline.from_pretrained(local_model_path)
    else:
        print("Local model not found, downloading from Hugging Face...")
        # Fallback to downloading from Hugging Face
        pipe = StableDiffusionPipeline.from_pretrained(huggingface_model)
    
    # Configure device (MPS for Mac M1/M2, CUDA for NVIDIA GPUs, CPU as fallback)
    if torch.backends.mps.is_available():
        device = "mps"
        print("Using MPS (Apple Silicon GPU)")
    elif torch.cuda.is_available():
        device = "cuda"
        print("Using CUDA (NVIDIA GPU)")
    else:
        device = "cpu"
        print("Using CPU (slower generation)")
    
    pipe.to(device)
    return pipe

pipe = setup_pipeline(config)

Loading Stable Diffusion model...
Using local model from: /Users/suvz47/Code/Cursor/NIAT Demo/models/image_generation/models--runwayml--stable-diffusion-v1-5/snapshots/451f4fe16113bff5a5d2269ed5ad43b0592e9a14


Loading pipeline components...: 100%|██████████| 7/7 [00:00<00:00, 20.39it/s]


Using MPS (Apple Silicon GPU)


## Enter a Prompt for Image Generation

In [5]:
prompt = "a futuristic city with flying cars"  # edit the prompt
print(f"Using prompt: {prompt}")

Using prompt: a futuristic city with flying cars


## Generate Image

In [6]:
def generate_image(pipe, prompt):
    print(f"🎯 Generating image for: '{prompt}'")
    result = pipe(prompt)
    return result.images[0]

image = generate_image(pipe, prompt)
image.show()

🎯 Generating image for: 'a futuristic city with flying cars'


 16%|█▌        | 8/50 [00:02<00:15,  2.79it/s]


KeyboardInterrupt: 

## Save the Generated Image

In [ ]:
def save_image(image, prompt, config):
    """
    Save the generated image in the output folder with prompt name.
    
    Args:
        image: PIL Image to save
        prompt (str): Original prompt used for generation
        config: Configuration object from config.ini
    
    Returns:
        str: Path to the saved image file
    """
    # Get output directory from config
    output_dir = config.get('Output', 'output_dir')
    max_filename_length = config.getint('Output', 'max_filename_length')
    
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Create a safe filename from the prompt
    safe_prompt = "".join(c for c in prompt if c.isalnum() or c in (' ', '-', '_')).rstrip()
    safe_prompt = safe_prompt.replace(' ', '_')[:max_filename_length]  # Use config length
    
    # Add timestamp for uniqueness
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"{safe_prompt}_{timestamp}.png"
    
    # Full path to save the image
    filepath = os.path.join(output_dir, filename)
    
    # Save the image
    image.save(filepath)
    print(f"💾 Image saved as: {filepath}")
    
    return filepath